In [2]:
%load_ext autoreload
%autoreload 2
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from transformers import AutoTokenizer, AutoModel
from sentence_transformers import SentenceTransformer
import torch
from lib.dataset import *   
from lib.data_handler import *
from lib.embeddings import *
from lib.vector_store import *
from lib.RAG_pipeline import *
from lib.utils import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
dataset_manager = FinanceRAGDataset("../data")
print("Available datasets:", dataset_manager.list_datasets())

Available datasets: ['ConvFinQA', 'FinQA', 'MultiHeritt', 'TATQA']


In [4]:
def get_random_sample(corpus_df):
    random_corpus = corpus_df.sample(n=1)
    pd.set_option('display.max_colwidth', None)
    return random_corpus['text'].values[0]

## Timing 

In [5]:
for DATASET_NAME in dataset_manager.list_datasets():
    what = "rephrase"
    timing_data = np.load(rf"C:\Users\alice\Desktop\UNIBO\2_Anno\2_semestre\NLP-torroni\PW\FinanceRag\data\{DATASET_NAME}\timing\timing_queries_{DATASET_NAME}_{what}.npy", allow_pickle=True).item()
    print(f"Length of timing_{DATASET_NAME}_{what}:", len(timing_data))
    average_time = np.mean(list(timing_data.values()))
    print(f"\t{round(average_time, 4)}")
    # print(timing_data)

Length of timing_ConvFinQA_rephrase: 506
	14.0514
Length of timing_FinQA_rephrase: 547
	13.7588
Length of timing_MultiHeritt_rephrase: 292
	14.434
Length of timing_TATQA_rephrase: 498
	13.4923


## Save npy of all query to expand

In [6]:
for DATASET_NAME in dataset_manager.list_datasets():
    corpus, queries, qrels = dataset_manager.load_dataset(DATASET_NAME)
    corpus, queries = reduce_dataset_size(corpus, queries, qrels)
    print("Dataset:", DATASET_NAME)

    all_queries = {}
    for id_query, query in queries.items():
        all_queries[id_query] = query

    # Save all queries for the dataset in one .npy file
    np.save(f"../data/{DATASET_NAME}/extracted_queries_{DATASET_NAME}.npy", all_queries)

    print(f"Done ({len(all_queries)}) queries\n")

Dataset: ConvFinQA
Done (126) queries

Dataset: FinQA
Done (344) queries

Dataset: MultiHeritt
Done (292) queries

Dataset: TATQA
Done (498) queries



## Get a random query, with relative corpus target

In [7]:
for DATASET_NAME in dataset_manager.list_datasets():
    corpus, queries, qrels = dataset_manager.load_dataset(DATASET_NAME)
    corpus, queries = reduce_dataset_size(corpus, queries, qrels)
    print("Dataset:", DATASET_NAME, "\n")

    # print(queries)
    for id_query, query in queries.items():
        print(f"Query id: {id_query}\t Query: {query}")
                
        answer_doc_id = qrels[qrels['query_id'] == id_query]['corpus_id'].tolist()[0]

        answer_doc = corpus[answer_doc_id][0]
        print(f"\nAnswer doc: id {answer_doc_id}\n{answer_doc}")

        break
    break

Dataset: ConvFinQA 

Query id: qd496f102	 Query: what was the average revenue from discontinued operations in 2013?

Answer doc: id dd496f152
dish network corporation notes to consolidated financial statements - continued 9 .
acquisitions dbsd north america and terrestar transactions on march 2 , 2012 , the fcc approved the transfer of 40 mhz of aws-4 wireless spectrum licenses held by dbsd north america and terrestar to us .
on march 9 , 2012 , we completed the dbsd transaction and the terrestar transaction , pursuant to which we acquired , among other things , certain satellite assets and wireless spectrum licenses held by dbsd north america and terrestar .
in addition , during the fourth quarter 2011 , we and sprint entered into a mutual release and settlement agreement ( the 201csprint settlement agreement 201d ) pursuant to which all issues then being disputed relating to the dbsd transaction and the terrestar transaction were resolved between us and sprint , including , but not l

See the query and the query expansions

In [17]:
import random

for DATASET_NAME in dataset_manager.list_datasets():
    corpus, queries, qrels = dataset_manager.load_dataset(DATASET_NAME)
    corpus, queries = reduce_dataset_size(corpus, queries, qrels)
    print("Dataset:", DATASET_NAME, "\n")
    
    qr = np.load(f"../data/{DATASET_NAME}/queries_expanded_{DATASET_NAME}_rephrase.npy", allow_pickle=True).item()
    q2d = np.load(f"../data/{DATASET_NAME}/queries_expanded_{DATASET_NAME}_Q2D.npy", allow_pickle=True).item()

    random_key = random.choice(list(queries.keys()))

    print(f"Query id: {random_key}\t Query: {queries[random_key]}\n")
    print(f"Rephrase query: {qr[random_key]}\n")
    print(f"Query to documents: {q2d[random_key]}\n")
    print("\n\n")

Dataset: ConvFinQA 

Query id: qd4975fb6	 Query: what is the sum of the weighted average common shares outstanding for diluted computations in 2017 and 2016?

Rephrase query: What is the sum of the weighted average number of common shares outstanding for diluted computations, taking into account all potential shares that could be issued in the future, for the years 2017 and 2016? This calculation should consider the potential impact of all share options, warrants, convertible securities, and other similar instruments that could increase the number of shares outstanding, and should be based on the actual number of shares issued during these years.

Query to documents: 'The sum of the weighted average common shares outstanding for diluted computations in 2017 and 2016 refers to the calculation of the number of shares that a company would have if all its potential shares were issued and outstanding. This calculation takes into account the effect of stock options, warrants, convertible bon

## Check how many tables per dataset

In [8]:
for DATASET_NAME in dataset_manager.list_datasets():
    corpus, queries, qrels = dataset_manager.load_dataset(DATASET_NAME)
    corpus, queries = reduce_dataset_size(corpus, queries, qrels)
    print(f"Dataset: {DATASET_NAME} has {len(corpus)} documents in corpus")

    multi_table_docs = {'1': 0, '2': 0, '3': 0, '4': 0, '+':0}  # Initialize the keys with 0
    for k,v in corpus.items():
        text = v[0]
        tables = extract_tables(v[0])
        if tables:
            if len(tables)==1:
                multi_table_docs['1'] += 1
            if len(tables)==2:
                multi_table_docs['2'] += 1
            if len(tables)==3:
                multi_table_docs['3'] += 1
            if len(tables)==4:
                multi_table_docs['4'] += 1
            if len(tables)>4:
                multi_table_docs['+'] += 1

    # unique print with all the information
    print(f"""\t{multi_table_docs['1']} documents with 1 table
          {multi_table_docs['2']} documents with 2 tables
          {multi_table_docs['3']} documents with 3 tables
          {multi_table_docs['4']} documents with 4 tables 
          {multi_table_docs['+']} documents with more than 4 tables\n""")


Dataset: ConvFinQA has 101 documents in corpus
	101 documents with 1 table
          0 documents with 2 tables
          0 documents with 3 tables
          0 documents with 4 tables 
          0 documents with more than 4 tables

Dataset: FinQA has 247 documents in corpus
	247 documents with 1 table
          0 documents with 2 tables
          0 documents with 3 tables
          0 documents with 4 tables 
          0 documents with more than 4 tables

Dataset: MultiHeritt has 876 documents in corpus
	491 documents with 1 table
          46 documents with 2 tables
          7 documents with 3 tables
          6 documents with 4 tables 
          1 documents with more than 4 tables

Dataset: TATQA has 248 documents in corpus
	248 documents with 1 table
          0 documents with 2 tables
          0 documents with 3 tables
          0 documents with 4 tables 
          0 documents with more than 4 tables



## Check how long are these tables

In [9]:
for DATASET_NAME in dataset_manager.list_datasets():
    corpus, queries, qrels = dataset_manager.load_dataset(DATASET_NAME)
    corpus, queries = reduce_dataset_size(corpus, queries, qrels)
    print(f"Dataset: {DATASET_NAME} has {len(corpus)} documents in corpus and {len(queries)} queries")

    tables_len = []
    text_with_tables_len = []
    percentages = []
    for k, v in corpus.items():
        text = v[0]
        len_text = len(text)
        tables = extract_tables(v[0])
        sum_len_table = 0
        if tables:
            for table in tables:
                tables_len.append(len(table))
                sum_len_table += len(table)
        text_without_tables_len = len_text - sum_len_table
        text_with_tables_len.append(text_without_tables_len)
        if text_without_tables_len > 0:
            percentages.append((sum_len_table / text_without_tables_len) * 100)
    
    len_table = np.mean(tables_len)
    print(f"Average length of tables: {int(len_table)}")
    text = np.mean(text_with_tables_len)
    print(f"Average length of text without tables: {int(text)}")
    print((len_table / text)*100)
    
    # average percentage of table out of the total without the tables
    print(f"Average percentage of table out of the total without the tables: {round(np.mean(percentages), 2)}%")
    print()

Dataset: ConvFinQA has 101 documents in corpus and 126 queries
Average length of tables: 471
Average length of text without tables: 3821
12.334825731430525
Average percentage of table out of the total without the tables: 15.7%

Dataset: FinQA has 247 documents in corpus and 344 queries
Average length of tables: 462
Average length of text without tables: 3823
12.091663481259749
Average percentage of table out of the total without the tables: 16.21%

Dataset: MultiHeritt has 876 documents in corpus and 292 queries
Average length of tables: 722
Average length of text without tables: 2350
30.75461852148205
Average percentage of table out of the total without the tables: 51.76%

Dataset: TATQA has 248 documents in corpus and 498 queries
Average length of tables: 491
Average length of text without tables: 1766
27.83001961066327
Average percentage of table out of the total without the tables: 36.07%



## check how many chunks there are per corpus per dataset

In [10]:
text_processor = DataHandler(Tokenizer(AutoTokenizer.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")),
                             Embedder(AutoModel.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")))
EMBEDDING_DIM = 384
MODEL_INPUT_SIZE = 256

In [11]:
for DATASET_NAME in dataset_manager.list_datasets():
    corpus, queries, qrels = dataset_manager.load_dataset(DATASET_NAME)
    corpus, queries = reduce_dataset_size(corpus, queries, qrels)
    print(f"Dataset: {DATASET_NAME} \tNumber of queries: {len(queries)}\t Number of documents: {len(corpus)}")

    pipeline = RAGPipeline(corpus, queries, qrels, text_processor)
    num_chunks = []
    for idx, text in corpus.items():
        data_obj = text_processor.load_data(text)
        data_obj = text_processor.tokenize()
        chunks = chunker(data_obj.data, max_length=MODEL_INPUT_SIZE, padding_value=0, overlap_percent=15)
        num_chunks.append(len(chunks))
    
    print(f"Average number of chunks: {round(np.mean(num_chunks),4)}\n")

Token indices sequence length is longer than the specified maximum sequence length for this model (1801 > 512). Running this sequence through the model will result in indexing errors


Dataset: ConvFinQA 	Number of queries: 126	 Number of documents: 101
Average number of chunks: 4.4752

Dataset: FinQA 	Number of queries: 344	 Number of documents: 247
Average number of chunks: 4.4291

Dataset: MultiHeritt 	Number of queries: 292	 Number of documents: 876
Average number of chunks: 3.3105

Dataset: TATQA 	Number of queries: 498	 Number of documents: 248
Average number of chunks: 2.621



In [12]:
import plotly.graph_objects as go
import plotly.subplots as sp

def plot_count_words(corpus_df, queries_df):
    # Calculate text lengths based on word count
    corpus_df['word_count'] = corpus_df['text'].apply(lambda x: len(x.split()))
    queries_df['word_count'] = queries_df['text'].apply(lambda x: len(x.split()))

    # Calculate max and average word counts
    max_word_count_corpus = corpus_df['word_count'].max()
    average_word_count_corpus = round(corpus_df['word_count'].mean(), 2)
    max_word_count_queries = queries_df['word_count'].max()
    average_word_count_queries = round(queries_df['word_count'].mean(), 2)

    print(f"Max word count: {max_word_count_corpus}")
    print(f"Average word count: {average_word_count_corpus}")
    print(f"Max word count in queries: {max_word_count_queries}")
    print(f"Average word count in queries: {average_word_count_queries}")

    # Create subplots
    fig = sp.make_subplots(rows=1, cols=2, subplot_titles=("Corpus Word Count Distribution", "Queries Word Count Distribution"))

    # Histogram for corpus
    fig.add_trace(go.Histogram(
        x=corpus_df['word_count'],
        nbinsx=50,
        marker_color='blue',
        name="Corpus Word Count"
    ), row=1, col=1)

    # Histogram for queries
    fig.add_trace(go.Histogram(
        x=queries_df['word_count'],
        nbinsx=50,
        marker_color='green',
        name="Queries Word Count"
    ), row=1, col=2)

    # Update layout
    fig.update_layout(
        title_text="Word Count Distribution in Corpus and Queries",
        showlegend=False,
        height=400,
        width=800
    )

    fig.update_xaxes(title_text="Word Count")
    fig.update_yaxes(title_text="Frequency")

    fig.show()

## Word count in the datasets

In [13]:
for DATASET_NAME in dataset_manager.list_datasets():
    print("========= DATASET:", DATASET_NAME, " =========\n")
    corpus, queries, qrels = dataset_manager.load_dataset(DATASET_NAME)
    print(f"Starting number of documents: {len(corpus)} and number of queries: {len(queries)}")

    corpus, queries = reduce_dataset_size(corpus, queries, qrels)
    print(f"After reducing: number of documents: {len(corpus)} and number of queries: {len(queries)}\n")

    corpus_df = pd.DataFrame(list(corpus.items()), columns=["id", "text"])
    corpus_df["text"] = corpus_df["text"].apply(lambda x: x[0] if isinstance(x, list) and len(x) > 0 else "")
    queries_df = pd.DataFrame(list(queries.items()), columns=["id", "text"])
    plot_count_words(corpus_df, queries_df)

========= DATASET: ConvFinQA  =========

Starting number of documents: 2066 and number of queries: 421
After reducing: number of documents: 101 and number of queries: 126

Max word count: 1586
Average word count: 685.39
Max word count in queries: 35
Average word count in queries: 13.89


========= DATASET: FinQA  =========

Starting number of documents: 2789 and number of queries: 1147
After reducing: number of documents: 247 and number of queries: 344

Max word count: 1661
Average word count: 685.09
Max word count in queries: 43
Average word count in queries: 16.71


========= DATASET: MultiHeritt  =========

Starting number of documents: 10475 and number of queries: 974
After reducing: number of documents: 876 and number of queries: 292

Max word count: 2654
Average word count: 474.03
Max word count in queries: 46
Average word count in queries: 17.97


========= DATASET: TATQA  =========

Starting number of documents: 2756 and number of queries: 1663
After reducing: number of documents: 248 and number of queries: 498

Max word count: 1100
Average word count: 287.44
Max word count in queries: 32
Average word count in queries: 12.35


## check the summaries of the tables

In [5]:
import re
import pandas as pd

def get_table(text):
    """Estrae le tabelle da un testo e le formatta in modo leggibile."""
    table_pattern = r'(?:(?:\n|\A)([^\n]+\|[^\n]+\n)((?:[-]+\|[-]+\n)?)((?:[^\n]+\|[^\n]+\n)+))'
    matches = re.findall(table_pattern, text)

    formatted_tables = []
    for match in matches:
        header, separator, rows = match
        columns = header.strip().split("|")  # Estrai colonne
        rows = [row.strip().split("|") for row in rows.strip().split("\n")]  # Estrai righe

        # Pulizia degli spazi
        columns = [col.strip() for col in columns]
        rows = [[cell.strip() for cell in row] for row in rows]

        # Creazione DataFrame
        df = pd.DataFrame(rows, columns=columns)
        
        # Aggiungi tabella formattata alla lista
        formatted_tables.append(df.to_markdown(index=False))

    return "\n\n".join(formatted_tables)  # Unisce le tabelle con spazi tra loro


In [6]:
import random

for DATASET_NAME in dataset_manager.list_datasets():
    print("========= DATASET:", DATASET_NAME, " =========")
    corpus, queries, qrels = dataset_manager.load_dataset(DATASET_NAME)
    corpus, queries = reduce_dataset_size(corpus, queries, qrels)

    extracted_tables = np.load(f"../data/{DATASET_NAME}/extracted_tables_{DATASET_NAME}.npy", allow_pickle=True).item()
    summaries_short = np.load(f"../data/{DATASET_NAME}/table_summaries_{DATASET_NAME}_short.npy", allow_pickle=True).item()
    summaries_long = np.load(f"../data/{DATASET_NAME}/table_summaries_{DATASET_NAME}_long.npy", allow_pickle=True).item()

    random_key = random.choice(list(extracted_tables.keys()))
    print(f"Selected Document ID: {random_key}")

    # Cerca il documento nel corpus con la chiave
    text = corpus[random_key][0]
    table = get_table(text)

    print(f"\nTable extracted:\n{extracted_tables[random_key][0]}\n")
    print(f"Short summary :\n\t{summaries_short.get(random_key, 'N/A')}\n")
    print(f"Long summary seconds: \n\t{summaries_long.get(random_key, 'N/A')}")
    break  

========= DATASET: ConvFinQA  =========
Selected Document ID: dd497ad9a

Table extracted:
Columns: in millions | 2007 | 2006 | 2005
Rows:
---------------- | ------ | ------ | ------
sales | $ 5245 | $ 4925 | $ 4625
operating profit | $ 501 | $ 399 | $ 219


Short summary :
	Financial Performance Over Three Years. This table shows the financial performance of a company over three years. Key trends include a steady increase in sales, a significant increase in operating profit, and a substantial increase in net profit. sales,  'operating profit,  'net profit,  'financial performance,  'trends,  'increase

Long summary seconds: 
	Financial Performance Over Three Years. The table displays the financial performance of a company over a three-year period, from 2005 to 2007. The key financial metrics include sales, operating profit, and net profit, all presented in millions. The table shows a steady increase in sales, operating profit, and net profit over the three-year period. sales,  'operati